In [1]:
import os
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

from torch.cuda.amp import GradScaler, autocast
from torchvision import transforms, models
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'


In [2]:
train_img_dir = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\images"
val_img_dir = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\images"


In [3]:
df = pd.read_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\fairface_label_train_cleaned.csv')
df


,file,age,gender,race,service_test
0,1.jpg,50-59,Male,East Asian,True
1,2.jpg,30-39,Female,Indian,False
2,3.jpg,3-9,Female,Black,False
3,4.jpg,20-29,Female,Indian,True
4,5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
85897,86739.jpg,20-29,Male,Middle Eastern,True
85898,86740.jpg,20-29,Male,Indian,True
85899,86741.jpg,10-19,Male,Indian,True
85900,86743.jpg,10-19,Female,Black,True


In [4]:
df = df[~df['age'].isin(['0-2', '3-9', '60-69'])]
df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\913567728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


In [5]:
age_mapping = {
    '10-19': 0,
    '20-29': 1,
    '30-39': 2,
    '40-49': 3,
    '50-59': 4,
}

df['age'] = df['age'].map(age_mapping).astype(int)

race_mapping = {
    'East Asian': 0,
    'Indian': 1,
    'Black': 2,
    'White': 3,
    'Middle Eastern': 4,
    'Latino_Hispanic': 5
}

#numerical labels
df['race'] = df['race'].map(race_mapping)


gender_mapping = {
    'Male': 0,
    'Female': 1
}

#  binary 
df['gender'] = df['gender'].map(gender_mapping)

train_df = df.drop(columns=['service_test'])

C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\3960336617.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].map(age_mapping).astype(int)
C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\3960336617.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].map(race_mapping)
C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\3960336617.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [6]:
train_df.head()

,file,age,gender,race
0,1.jpg,4,0,0
1,2.jpg,2,1,1
3,4.jpg,1,1,1
4,5.jpg,1,1,1
5,6.jpg,1,0,3


In [7]:
train_df.to_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\fairface_label_train_final.csv', index=False)

In [8]:
df = pd.read_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\fairface_label_val_cleaned.csv')
df

,file,age,gender,race,service_test
0,1.jpg,3-9,Male,East Asian,False
1,2.jpg,50-59,Female,East Asian,True
2,3.jpg,30-39,Male,White,True
3,4.jpg,20-29,Female,Latino_Hispanic,True
4,5.jpg,20-29,Male,Southeast Asian,False
...,...,...,...,...,...
10831,10950.jpg,30-39,Male,White,True
10832,10951.jpg,50-59,Male,White,False
10833,10952.jpg,60-69,Male,Latino_Hispanic,False
10834,10953.jpg,20-29,Female,East Asian,False


In [9]:
df = df[~df['age'].isin(['0-2', '3-9', '60-69'])]
df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\913567728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


In [10]:
age_mapping = {
    '10-19': 0,
    '20-29': 1,
    '30-39': 2,
    '40-49': 3,
    '50-59': 4,
}


df['age'] = df['age'].map(age_mapping).astype(int)

race_mapping = {
    'East Asian': 0,
    'Indian': 1,
    'Black': 2,
    'White': 3,
    'Middle Eastern': 4,
    'Latino_Hispanic': 5
}

df['race'] = df['race'].map(race_mapping)


gender_mapping = {
    'Male': 0,
    'Female': 1
}

df['gender'] = df['gender'].map(gender_mapping)

val_df = df.drop(columns=['service_test'])



C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\2122856892.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].map(age_mapping).astype(int)
C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\2122856892.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].map(race_mapping)
C:\Users\shrit\AppData\Local\Temp\ipykernel_42912\2122856892.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [11]:
val_df.head()

,file,age,gender,race
1,2.jpg,4,1,0
2,3.jpg,2,0,3
3,4.jpg,1,1,5
4,5.jpg,1,0,0
5,6.jpg,2,0,5


In [12]:
val_df.to_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\fairface_label_val_final.csv', index=False)

In [13]:
train_df['gender'] = train_df['gender'].astype(float)
val_df['gender'] = val_df['gender'].astype(float)

In [14]:
train_image_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\images"
val_image_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\images"

In [ ]:
class MultiTaskModel(nn.Module):
    def __init__(self, device="cuda"):
        super(MultiTaskModel, self).__init__()
        #  pretrained EfficientNet as transfer learning
        self.backbone = models.efficientnet_b0(pretrained=True)  # Use B0 variant bc its fast
        self.backbone.classifier = nn.Identity()  # remove classification layer
        
        # add custom heads for multi-task 
        self.age_head = nn.Sequential(
    nn.Dropout(0.2),  # dropout
    nn.Linear(1280, 5)
    )
        self.gender_head = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1280, 1)
    )
        self.race_head = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1280, 6)
    )

    def forward(self, x):
        features = self.backbone(x)  # features from the EfficientNet 
        age_out = self.age_head(features)
        gender_out = self.gender_head(features)
        race_out = self.race_head(features)
        return age_out, gender_out, race_out


In [ ]:
class FairFaceDataset(Dataset): #transforming the dataset to make it work
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        # Extract labels
        age = self.data.iloc[idx, 1]
        gender = self.data.iloc[idx, 2]
        race = self.data.iloc[idx, 3]

        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(age, dtype=torch.long), torch.tensor(gender, dtype=torch.float), torch.tensor(race, dtype=torch.long)


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),  #  EfficientNet input size
    transforms.RandomHorizontalFlip(), # augmentations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [18]:
train_dataset = FairFaceDataset(dataframe=train_df, root_dir=train_image_path, transform=train_transform)
val_dataset = FairFaceDataset(dataframe=val_df, root_dir=val_image_path, transform=val_transform)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False,pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskModel(device=device).to(device) #useing GPU

c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion_age = nn.CrossEntropyLoss()
criterion_gender = nn.BCEWithLogitsLoss()  # Use raw 
criterion_race = nn.CrossEntropyLoss()

In [ ]:
print("Unique age labels:", train_df['age'].unique())
print("Unique race labels:", train_df['race'].unique()) # testing age labels

Unique age labels: [4 2 1 3 0]
Unique race labels: [0 1 3 4 5 2]


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MultiTaskModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (sca

In [24]:
optimizer = torch.optim.Adam([
    {'params': model.backbone.parameters(), 'lr': 1e-6},  # Smaller LR for pre-trained backbone
    {'params': model.age_head.parameters(), 'lr': 1e-4},  # Larger LR for new heads
    {'params': model.gender_head.parameters(), 'lr': 1e-4},
    {'params': model.race_head.parameters(), 'lr': 1e-4},
])

In [25]:
alpha, beta, gamma = 1.5, 1.0, 1.5

In [26]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    correct_age, correct_gender, correct_race = 0, 0, 0
    total_samples = 0

    scaler = torch.amp.GradScaler()
    progress_bar = tqdm(dataloader, desc="Training", unit="batch")

    for images, age_labels, gender_labels, race_labels in progress_bar:
        images, age_labels, gender_labels, race_labels = (
            images.to(device),
            age_labels.to(device),
            gender_labels.to(device),
            race_labels.to(device),
        )

        optimizer.zero_grad()

        # Use autocast for mixed precision operations
        with torch.amp.autocast(device_type='cuda'):
            age_out, gender_out, race_out = model(images)

            # Compute losses
            loss_age = criterion_age(age_out, age_labels)
            loss_gender = criterion_gender(gender_out.squeeze(), gender_labels)
            loss_race = criterion_race(race_out, race_labels)

            # Combine losses
            loss = alpha * loss_age + beta * loss_gender + gamma * loss_race

        # Backward pass using scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Update total loss
        total_loss += loss.item()

        # Compute predictions and accuracy
        age_preds = torch.argmax(age_out, dim=1)
        gender_preds = torch.round(torch.sigmoid(gender_out.squeeze()))
        race_preds = torch.argmax(race_out, dim=1)

        correct_age += (age_preds == age_labels).sum().item()
        correct_gender += (gender_preds == gender_labels).sum().item()
        correct_race += (race_preds == race_labels).sum().item()

        total_samples += age_labels.size(0)

        progress_bar.set_postfix({
            "Loss": loss.item(),
            "Age Acc": f"{(correct_age / total_samples) * 100:.2f}%",
            "Gender Acc": f"{(correct_gender / total_samples) * 100:.2f}%",
            "Race Acc": f"{(correct_race / total_samples) * 100:.2f}%"
        })

    # Overall accuracy
    age_acc = correct_age / total_samples
    gender_acc = correct_gender / total_samples
    race_acc = correct_race / total_samples

    return total_loss / len(dataloader), age_acc, gender_acc, race_acc

def validate(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct_age, correct_gender, correct_race = 0, 0, 0
    total_samples = 0

    progress_bar = tqdm(dataloader, desc="Validating", unit="batch")

    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda'):
            for images, age_labels, gender_labels, race_labels in progress_bar:
                images, age_labels, gender_labels, race_labels = (
                    images.to(device),
                    age_labels.to(device),
                    gender_labels.to(device),
                    race_labels.to(device),
                )
                age_out, gender_out, race_out = model(images)

                # Compute losses
                loss_age = criterion_age(age_out, age_labels)
                loss_gender = criterion_gender(gender_out.squeeze(), gender_labels)
                loss_race = criterion_race(race_out, race_labels)

                loss = loss_age + loss_gender + loss_race
                total_loss += loss.item()

                # Compute predictions and accuracy
                age_preds = torch.argmax(age_out, dim=1)
                gender_preds = torch.round(torch.sigmoid(gender_out.squeeze()))
                race_preds = torch.argmax(race_out, dim=1)

                correct_age += (age_preds == age_labels).sum().item()
                correct_gender += (gender_preds == gender_labels).sum().item()
                correct_race += (race_preds == race_labels).sum().item()

                total_samples += age_labels.size(0)

                progress_bar.set_postfix({
                    "Loss": loss.item(),
                    "Age Acc": f"{(correct_age / total_samples) * 100:.2f}%",
                    "Gender Acc": f"{(correct_gender / total_samples) * 100:.2f}%",
                    "Race Acc": f"{(correct_race / total_samples) * 100:.2f}%"
                })

    # Overall accuracy
    age_acc = correct_age / total_samples
    gender_acc = correct_gender / total_samples
    race_acc = correct_race / total_samples

    return total_loss / len(dataloader), age_acc, gender_acc, race_acc

In [27]:
best_model_path = "best_face_age_gender_race_extraction_state_dict2.pth"

# Variable to track the best validation loss
best_val_loss = float("inf")

num_epochs = 10
for epoch in range(num_epochs):
    # Unpack training and validation results
    train_loss, train_age_acc, train_gender_acc, train_race_acc = train_one_epoch(model, train_loader, optimizer, device)
    val_loss, val_age_acc, val_gender_acc, val_race_acc = validate(model, val_loader, device)

    # Print epoch performance
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Age Acc: {train_age_acc * 100:.2f}%, Gender Acc: {train_gender_acc * 100:.2f}%, Race Acc: {train_race_acc * 100:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Age Acc: {val_age_acc * 100:.2f}%, Gender Acc: {val_gender_acc * 100:.2f}%, Race Acc: {val_race_acc * 100:.2f}%")

    # Check if this is the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)  # Save only state_dict
        print(f"Best model state_dict saved at epoch {epoch+1} with validation loss: {val_loss:.4f}")

Training:   1%|          | 8/1109 [00:02<06:01,  3.05batch/s, Loss=5.83, Age Acc=20.31%, Gender Acc=51.95%, Race Acc=16.80%]


KeyboardInterrupt: 

In [ ]:
results = []  # List to store predictions and true labels
model = torch.load("best_face_age_gender_race_extraction_full_model_dict2.pth")

model.eval()
with torch.no_grad():
    for images, age_labels, gender_labels, race_labels in val_loader:  # or train_loader
        images, age_labels, gender_labels, race_labels = (
            images.to(device),
            age_labels.to(device),
            gender_labels.to(device),
            race_labels.to(device),
        )
        
        # Forward pass to get predictions
        age_out, gender_out, race_out = model(images)

        # Convert outputs to predictions
        age_preds = torch.argmax(age_out, dim=1).cpu().numpy()
        gender_preds = torch.round(torch.sigmoid(gender_out.squeeze())).cpu().numpy()
        race_preds = torch.argmax(race_out, dim=1).cpu().numpy()

        # Append results for each sample
        for i in range(len(age_labels)):
            results.append({
                "true_age": age_labels[i].item(),
                "predicted_age": age_preds[i],
                "true_gender": gender_labels[i].item(),
                "predicted_gender": gender_preds[i],
                "true_race": race_labels[i].item(),
                "predicted_race": race_preds[i],
                "age_confidence": torch.softmax(age_out[i], dim=0).cpu().numpy(),
                "race_confidence": torch.softmax(race_out[i], dim=0).cpu().numpy(),
                "gender_confidence": torch.sigmoid(gender_out[i].squeeze()).cpu().item(),
            })

NameError: name 'torch' is not defined

In [ ]:
age_misclassified = [r for r in results if r["true_age"] != r["predicted_age"]]
gender_misclassified = [r for r in results if r["true_gender"] != r["predicted_gender"]]
race_misclassified = [r for r in results if r["true_race"] != r["predicted_race"]]

print("Age Misclassified:", len(age_misclassified))
print("Gender Misclassified:", len(gender_misclassified))
print("Race Misclassified:", len(race_misclassified))

Age Misclassified: 4689
Gender Misclassified: 1227
Race Misclassified: 3849


In [ ]:
from collections import Counter

age_errors = Counter([r["true_age"] for r in age_misclassified])
gender_errors = Counter([r["true_gender"] for r in gender_misclassified])
race_errors = Counter([r["true_race"] for r in race_misclassified])

print("Age Misclassification by Class:", age_errors)
print("Gender Misclassification by Class:", gender_errors)
print("Race Misclassification by Class:", race_errors)

Age Misclassification by Class: Counter({2: 1610, 3: 960, 1: 940, 0: 755, 4: 424})
Gender Misclassification by Class: Counter({1.0: 823, 0.0: 404})
Race Misclassification by Class: Counter({5: 1034, 4: 714, 1: 640, 3: 604, 0: 510, 2: 347})


In [ ]:
# Print examples of misclassified samples for race
for r in race_misclassified[:10]:  # Limit to 5 examples
    print(f"True Race: {r['true_race']}, Predicted Race: {r['predicted_race']}, Confidence: {r['race_confidence']}")

True Race: 3, Predicted Race: 4, Confidence: [0.13106075 0.03898117 0.00345048 0.31210607 0.3698356  0.14456597]
True Race: 5, Predicted Race: 0, Confidence: [0.58087504 0.02934828 0.00495047 0.11416876 0.08435402 0.18630344]
True Race: 5, Predicted Race: 4, Confidence: [0.07642388 0.06654444 0.01266471 0.2466833  0.41997445 0.17770919]
True Race: 0, Predicted Race: 3, Confidence: [0.01512535 0.01880851 0.01108239 0.6046333  0.14544171 0.20490879]
True Race: 5, Predicted Race: 3, Confidence: [0.0574134  0.0388911  0.01510215 0.58865404 0.05463748 0.2453018 ]
True Race: 0, Predicted Race: 3, Confidence: [0.25892773 0.01099635 0.0202651  0.40851545 0.08500932 0.21628611]
True Race: 3, Predicted Race: 2, Confidence: [0.08347745 0.27981693 0.34190553 0.08170161 0.05467979 0.1584187 ]
True Race: 3, Predicted Race: 4, Confidence: [0.03970648 0.01477044 0.00052059 0.33934268 0.3843736  0.22128631]
True Race: 4, Predicted Race: 2, Confidence: [0.05819422 0.03573163 0.26837817 0.16592196 0.2148

In [ ]:
from torchvision import transforms
from PIL import Image
import torch

# Load the image
image_path = "C:\\Users\\shrit\\Desktop\\Ml_Projects\\Ml_Projects\\Data\\celeba\\img_align_celeba\\000001.jpg" # Replace with your image file path
image = Image.open(image_path).convert("RGB")

# Define preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for feature model
])

# Preprocess the image
image_tensor = preprocess(image)
image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension

In [ ]:
# Load your pretrained face feature extraction model
model = torch.load(r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Notebooks\best_face_age_gender_race_extraction_full_model.pth")  # Replace with your model path
model.eval()  # Set the model to evaluation mode

import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the same device
model = model.to(device)

# Move the input tensor to the same device
image_tensor = image_tensor.to(device)

# Run the model
with torch.no_grad():
    output = model(image_tensor)

Using device: cuda


C:\Users\shrit\AppData\Local\Temp\ipykernel_41464\3722254749.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(r"C:\Users\shrit\Desktop\Ml_Projects\Ml_P

In [ ]:
print(f"Output Values:\n{output}")

Output Values:
(tensor([[-1.2899,  1.1020,  0.1450, -1.1026, -2.5615]], device='cuda:0'), tensor([[6.7359]], device='cuda:0'), tensor([[-0.1154, -1.0456, -2.5352,  0.4176, -0.2297,  0.4673]],
       device='cuda:0'))


In [ ]:
age_group_logits = output[0]
predicted_age_group = torch.argmax(age_group_logits, dim=1).item()

# Example age mapping
age_mapping = {0: "10-19", 1: "20-29", 2: "30-39", 3: "40-49", 4: "50-59"}
age_group_label = age_mapping[predicted_age_group]
print(f"Predicted Age Group: {age_group_label}")



gender_logits = output[1]
gender_prob = torch.sigmoid(gender_logits).item()  # Probability of being "Female"

# Example gender mapping
gender_label = "Female" if gender_prob > 0.5 else "Male"
print(f"Predicted Gender: {gender_label} (Probability: {gender_prob:.2f})")


import torch.nn.functional as F

race_logits = output[2]
race_probs = F.softmax(race_logits, dim=1).squeeze(0).cpu().numpy()

# Example race mapping
race_mapping = {
    0: "East Asian", 1: "Indian", 2: "Black", 
    3: "White", 4: "Middle Eastern", 5: "Latino_Hispanic"
}

predicted_race_index = torch.argmax(race_logits, dim=1).item()
race_label = race_mapping[predicted_race_index]

print(f"Predicted Race: {race_label}")
print(f"Race Probabilities: {race_probs}")

Predicted Age Group: 20-29
Predicted Gender: Female (Probability: 1.00)
Predicted Race: Latino_Hispanic
Race Probabilities: [0.17035566 0.0672012  0.0151507  0.2902764  0.15194598 0.30507004]
